In [1]:
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
import pandas as pd
import os
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, GRU, Embedding, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Activation
from tensorflow.keras.optimizers import RMSprop, Adam, Nadam, SGD, Adadelta
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.backend import square, mean
from tensorflow.keras.models import load_model



In [3]:
#loss_mse_warmup函数功能是将去掉的一部分热启动所需消耗数据后的剩余数据进行mse评价
def loss_mse_warmup(y_true, y_pred):
    y_true_slice = y_true[:,warmup_steps:,:]
    y_pred_slice = y_pred[:,warmup_steps:,:]
    mse = mean(square(y_true_slice-y_pred_silce))
    return mse

In [4]:
def run_model_gru_sigmoid_1(num_x_events, num_y_events):
    model = Sequential()
    model.add(GRU(units=128, return_sequences=True,
                 input_shape=(None, num_x_events,)))
    model.add(BatchNormalization())
    model.add(GRU(units=64,return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dense(num_y_events, activation='sigmoid'))
    model.summary()
    return model

In [1]:
#批生成器
def batch_generator(batch_size, sequence_length):
    while True:
        x_shape = (batch_size, sequence_length, num_x_events)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)
        
        y_shape = (batch_size, sequence_length, num_y_events)
        y_batch =  np.zeros(shape=y_shape, dtype=np.float16)
        
        for i in range(batch_size):
            idx = np.random.randint(num_train - sequence_length)
            
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
            
        yield(x_batch, y_batch)

In [2]:
generator = batch_generator(batch_size=batch_size, sequence_length=sequence_length)
validation_data = (np.expand_dims(x_test_scaled, axis=0), 
                  np.expand_dims(y_test_scaled, axis=0))

perform_indics = model.fit(x=generator,
                          epochs=epochs,
                          steps_per_epoch=steps_per_epoch,
                          validation_data=validation_data,
                          callbacks=keras_callbacks,
                          verbose=2)

NameError: name 'batch_size' is not defined

In [3]:
#将训练时打包处理的数据生成的片段历史记录重新连接起来变成连续完成的时间序列
def collect_hist(accum_hist, next_hist):
    keys = list(next_hist.keys())
    for k in keys:
        if k in accum_hist:
            accum_hist[k].extend(next_hist[k])
        else:
            accum_hist[k] = next_hist[k]
            
        return accum_hist

In [ ]:
def plot_hist(h, xsize=6, ysize=5):
    fig_size=plt.rcParams['figure.figsize']
    plt.rcParams['figure.figsize'] =[xsize, ysize]
    
    ks = list(h.keys())
    n2 = math.floor(len(ks)/2)
    train_keys = ks[0:n2]
    valid_keys = ks[n2:2*n2]
    
    for i in range(n2):
        plt.plot(h[train_keys[i]])
        plt.plot(h[valid_keys[i]])
        plt.title("Training vs Validation" + train_keys[i])
        plt.ylabel(train_keys[i])
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.draw()
        plt.show()
        
    return 